In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17049,2024-12-25,Eua Ncaa,15:30,Murray State,Loyola (Ill).,2.05,1.80,138.5,1.80,1.95,1.5,1.95,1.80,xQ57sio2,0.487805,0.555556,0.555556,0.512821,0.043360,132.456,38.545503,0.291006,0.6,1.341641,2.236068,123.30,1.956,0.948093,0.484710,-16.0,104.204,26.595644,0.255227,1.2,1.643168,1.369306,110.84,1.488,0.467247,0.314010,4.0,90,68,1.37,1.63,106.128,127.302,0.091832,0.056569,0.056569,-1.36,-0.272,-3.860294,0.723394,0.6,-0.123394,9.000000e-02,1.800000e-02,4.444444e+01,0.513534,0.7,0.186466
17050,2024-12-25,Eua Ncaa,18:00,Charlotte,Coll Of Charltn,3.17,1.38,144.5,1.80,1.95,5.5,1.95,1.77,UJhg5fw9,0.315457,0.724638,0.555556,0.512821,0.040095,199.940,94.734077,0.473813,1.8,1.643168,0.912871,309.26,2.638,1.134711,0.430141,2.0,167.236,61.264345,0.366335,2.4,1.341641,0.559017,183.26,2.246,0.850782,0.378799,28.0,94,77,3.29,2.38,126.346,157.402,0.556361,0.056569,0.068430,0.66,0.132,16.439394,0.715735,0.6,-0.115735,3.380000e+00,6.760000e-01,5.621302e-01,0.550310,0.8,0.249690
17051,2024-12-25,Eua Ncaa,20:30,Hawaii,Oakland,1.67,2.28,133.5,1.91,1.91,-4.5,2.05,1.71,Qon23Y7L,0.598802,0.438596,0.523560,0.523560,0.037399,229.510,210.678818,0.917950,1.2,1.643168,1.369306,244.20,3.394,2.983661,0.879099,-2.0,322.512,247.678573,0.767967,0.6,1.341641,2.236068,444.00,5.016,4.243940,0.846081,-57.0,55,74,4.44,6.00,240.636,417.290,0.218398,0.000000,0.127881,-0.71,-0.142,-4.718310,0.580829,0.8,0.219171,-1.630000e+00,-3.260000e-01,-3.926380e+00,0.322893,0.4,0.077107
17052,2024-12-25,Eua Ncaa,23:00,Nebraska,Oregon State,1.61,2.42,136.5,1.80,1.95,-5.5,2.05,1.71,f1xYcCVr,0.621118,0.413223,0.555556,0.512821,0.034341,117.236,35.994800,0.307029,2.4,1.341641,0.559017,84.87,1.690,0.887046,0.524879,42.0,95.546,14.115243,0.147732,3.0,0.000000,0.000000,89.60,1.272,0.296766,0.233307,80.0,69,80,1.23,1.12,116.548,223.230,0.284246,0.056569,0.127881,-0.28,-0.056,-10.892857,0.756840,0.9,0.143160,-2.470000e+00,-4.940000e-01,-2.874494e+00,0.358984,0.3,-0.058984
17053,2024-12-25,Filipinas Copa Commissioners,08:30,Ginebra Kings,Magnolia Hotshots,1.58,2.32,190.5,1.87,1.86,-4.5,1.99,1.73,Cj2hDUDE,0.632911,0.431034,0.534759,0.537634,0.063946,152.672,32.117228,0.210368,1.8,1.643168,0.912871,118.56,1.592,0.419488,0.263497,27.0,167.962,35.043355,0.208639,0.6,1.341641,2.236068,148.50,1.702,0.334619,0.196604,-7.0,114,99,1.04,1.50,146.086,189.782,0.268338,0.003791,0.098843,0.88,0.176,3.295455,0.720292,0.8,0.079708,-1.290000e+00,-2.580000e-01,-5.116279e+00,0.647904,0.4,-0.247904
17054,2024-12-25,Israel Superliga,15:50,Maccabi Ironi Ramat Gan,Hapoel Tel-Aviv,3.69,1.26,172.5,1.87,1.86,7.5,1.99,1.73,E9J16IZj,0.271003,0.793651,0.534759,0.537634,0.064654,161.182,74.777873,0.463934,1.2,1.643168,1.369306,158.40,1.828,0.856720,0.468665,1.0,97.724,8.365945,0.085608,3.0,0.000000,0.000000,104.40,1.110,0.071764,0.064652,87.0,90,90,1.76,1.16,128.402,139.640,0.694250,0.003791,0.098843,0.68,0.136,19.779412,0.648341,0.5,-0.148341,0.000000e+00,0.000000e+00,inf,0.639521,0.6,-0.039521
17055,2024-12-25,Macedônia Do Norte Prva Liga,14:00,MKK Kumanovo,TFT Skopje,1.77,1.91,157.5,1.83,1.83,-1.5,1.83,1.83,xzgqiT72,0.564972,0.523560,0.546448,0.546448,0.088532,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,178.808,123.406767,0.690164,1.2,1.643168,1.369306,110.70,2.508,1.740264,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
17074,22:00,Eua Nba,Golden State Warriors,Los Angeles Lakers,1.59,Back Home
